In [105]:
import numpy as np
import pandas as pd
# 5- NDA-SE
# 20 - ACTO-A

In [106]:
my_test_result_list = np.load('results.np.npy')
print(my_test_result_list.shape)
filename = "data-prep-ema{}-beta{}.csv".format(10,99)
data = pd.read_csv(filename, parse_dates=["timestamp"])

(56, 258, 4)


In [107]:
ori_shape = my_test_result_list.shape



In [108]:
groups = data.set_index('timestamp').groupby(lambda x: x.date())

In [109]:
# start day is the learning period.
ori_shape = my_test_result_list.shape
my_test_result_list = my_test_result_list.reshape(int(ori_shape[0]/2), ori_shape[1]*2, ori_shape[2])
start_day = 30
i = 0
my_df_list = []
for index, df in groups:
    print("handling day: {}".format(i))
    if i >= start_day and i-start_day < len(my_test_result_list):
        print(i)
        my_df = df[['last_20','value_ema_10_beta_99_20','last_5','value_ema_10_beta_99_5']]
        print(len(my_test_result_list[i-start_day,:,0]))
        print(len(my_df))
        my_df['test_y_20'] = my_test_result_list[i-start_day,:,0]
        my_df['test_y_5'] = my_test_result_list[i - start_day,:,1] 
        my_df['valid_y_20'] = my_test_result_list[i - start_day,:,2]
        my_df['valid_y_5'] = my_test_result_list[i - start_day,:,3]
        my_df_list.append(my_df)
    i+=1

handling day: 0
handling day: 1
handling day: 2
handling day: 3
handling day: 4
handling day: 5
handling day: 6
handling day: 7
handling day: 8
handling day: 9
handling day: 10
handling day: 11
handling day: 12
handling day: 13
handling day: 14
handling day: 15
handling day: 16
handling day: 17
handling day: 18
handling day: 19
handling day: 20
handling day: 21
handling day: 22
handling day: 23
handling day: 24
handling day: 25
handling day: 26
handling day: 27
handling day: 28
handling day: 29
handling day: 30
30
516
516
handling day: 31
31
516
516
handling day: 32
32
516
516
handling day: 33
33
516
516
handling day: 34
34
516
516
handling day: 35
35
516
516
handling day: 36
36
516
516
handling day: 37
37
516
516
handling day: 38
38
516
516
handling day: 39
39
516
516
handling day: 40
40
516
516
handling day: 41
41
516
516
handling day: 42
42
516
516
handling day: 43
43
516
516
handling day: 44
44
516
516
handling day: 45
45
516
516
handling day: 46
46
516
516
handling day: 47
47
516


In [117]:
import GPy
import GPyOpt
from functools import partial

In [126]:
mixed_domain = [{'name': 'buy_threshold', 'type': 'continuous', 'domain': (0.0, 0.002)},
                 {'name': 'sell_threshold', 'type': 'continuous', 'domain': (-0.001, 0.0)},
                 {'name': 'min_hold_steps', 'type': 'discrete', 'domain': range(10,100)},
                 {'name': 'stop_loss', 'type': 'continuous', 'domain':(-0.03, -0.002)},
                 {'name': 'stop_gain', 'type': 'continuous', 'domain':(0.001, 0.03)}
         ]
    

    
def opt_wrapper(verbose, X_list):
    
    print(X_list)
    buy_threshold = X_list[0][0]
    sell_threshold = X_list[0][1]
    min_hold_steps = int(X_list[0][2])
    stop_loss = X_list[0][3]
    stop_gain = X_list[0][4]
    #stop_gain = int(X_list[0][4])
    tot_profit = 1
    tot_stock_profit = 1
    last_step = None
    max_trades = 3
    cost = 0.001
    hold=-1
    for day_idx in range(len(my_df_list)):
        #print("starting day {}".format(day_idx))
        trades = 0
        daily_profit = 1
        state = 0
        df = my_df_list[day_idx]
        for step in range(len(df)):
            if state == 0 and trades<max_trades:
                if df.iloc[step]['test_y_5'] > buy_threshold:
                    price = df.iloc[step]['last_5']
                    hold = 5
                    state = 1
                    last_step = step
                    if verbose:
                        print("buy at step {} price:{}".format(step, price))
                elif df.iloc[step]['test_y_20'] > buy_threshold:
                    price = df.iloc[step]['last_20']
                    hold = 20
                    if verbose:
                        print("buy at step {} price:{}".format(step, price))
                    state = 1
                    last_step = step
            elif state == 1:
                cur_price = df.iloc[step]['last_'+str(hold)]
                tmp_profit = (cur_price - price)/price

                if (df.iloc[step]['test_y_'+str(hold)] < buy_threshold and 
                    step - last_step > min_hold_steps) or step == len(df)-1 or \
                    tmp_profit < stop_loss or \
                    tmp_profit > stop_gain:
                    if verbose and tmp_profit < stop_loss:
                        print("STOP LOSS!!! cur:{} pri:{}, profit:{}, stop_loss:{}".format(cur_price, price,tmp_profit, stop_loss))
                    last_price = price
                    price = df.iloc[step]['last_'+str(hold)]
                    if verbose:
                        print("sell at step {} price:{}".format(step, price))
                    
                    tot_profit *= (1+tmp_profit-cost)
                    daily_profit *= (1 + tmp_profit - cost)
                    state = 0
                    hold = -1
                    trades += 1
        last_20 = df.iloc[len(df)-1].last_20
        open_20 = df.iloc[0].last_20
        stock_profit_20 = (last_20 - open_20)/ open_20
        
        last_5 = df.iloc[len(df)-1].last_5
        open_5 = df.iloc[0].last_5
        stock_profit_5 = (last_5 - open_5)/ open_5
        
        stock_profit = (stock_profit_20 + stock_profit_5) / 2
        tot_stock_profit *= (1+stock_profit)
        if verbose:
            print("finishing day {}, daily_profit:{}, stock_profit:{}".format(day_idx, daily_profit, stock_profit))
    print("{}, profit:{}, stock_profit: {}".format(X_list, tot_profit, tot_stock_profit))
    return -tot_profit
    

In [125]:
myBopt = GPyOpt.methods.BayesianOptimization(partial(opt_wrapper, False),  # Objective function       
                                     domain=mixed_domain,          # Box-constraints of the problem
                                     initial_design_numdata = 20,   # Number data initial design
                                     acquisition_type='EI',        # Expected Improvement
                                     exact_feval = True)           # True evaluations, no sample noise

myBopt.run_optimization(100,eps=0)

[[ 1.33851437e-03 -9.71473980e-04  8.20000000e+01 -2.53171993e-02
   6.87594039e-03]]
[[ 1.33851437e-03 -9.71473980e-04  8.20000000e+01 -2.53171993e-02
   6.87594039e-03]], profit:1.0609489791258713, stock_profit: 1.054564721918328
[[ 5.65437665e-04 -6.35077663e-06  3.70000000e+01 -2.99389879e-02
   1.41805142e-03]]


KeyboardInterrupt: 

In [127]:
opt_wrapper(True, np.array([[ 1.80538243e-03,-4.69608146e-04,6.40000000e+01,-2.58564345e-02,1.54588708e-02]]))

[[ 1.80538243e-03 -4.69608146e-04  6.40000000e+01 -2.58564345e-02
   1.54588708e-02]]
buy at step 4 price:81.73666666666666
sell at step 69 price:82.31
finishing day 0, daily_profit:1.0060143958239878, stock_profit:0.013606034219377665
buy at step 4 price:244.76666666666668
sell at step 69 price:245.15
finishing day 1, daily_profit:1.0005661173907123, stock_profit:0.0009285844499134135
buy at step 4 price:82.79666666666667
sell at step 69 price:83.16
finishing day 2, daily_profit:1.0033882603969564, stock_profit:0.0024554739147848313
buy at step 1 price:82.545
sell at step 66 price:83.42
finishing day 3, daily_profit:1.0096002786358957, stock_profit:0.013338512191536572
buy at step 118 price:249.35
sell at step 183 price:250.5
finishing day 4, daily_profit:1.0036119911770605, stock_profit:0.0032666880383962964
buy at step 1 price:83.99166666666667
sell at step 66 price:84.74
finishing day 5, daily_profit:1.0079096140490127, stock_profit:0.008642655505481057
buy at step 1 price:85.25166

-1.1144493050430835